# 텍스트 분류 실습 과제 노트북

## 과제 목표

하나의 텍스트 데이터셋(예: 리뷰, 댓글, SNS 글 등)에 대해

1. 텍스트 데이터를 로드하고 간단한 EDA(탐색적 데이터 분석)를 수행한다.
2. TF-IDF 벡터화를 적용하고,
3. 세 가지 모델을 학습 및 비교한다.
   - 로지스틱 회귀 (`LogisticRegression`)
   - 나이브 베이즈 (`MultinomialNB`)
   - 선형 SVM (`LinearSVC`)
4. 평가지표(특히 macro-F1)를 기준으로 모델 성능을 비교·분석한다.
5. 간단한 보고서(요약 문장)를 마크다운으로 정리한다.

---

>


In [1]:
# TODO: 필요한 라이브러리를 임포트하세요.
# 예시:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score

plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

print("라이브러리 임포트 완료")

라이브러리 임포트 완료


## 1. 데이터 불러오기 및 전처리

### 1-1. CSV 파일 불러오기




In [2]:
%pwd

'e:\\데이터분석가 부트캠프\\실습\\DataAnalysis_bootcamp\\5차시 실습(통계기반 자연어처리)'

In [ ]:
df = pd.read_csv('movie_reviews.csv', encoding='utf-8')
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


### 1-2. 간단 EDA

- 데이터 크기 확인 (`df.shape`)
- 레이블 분포 확인 (`value_counts()`)
- 결측값 여부 확인 (`isna().sum()`)


In [4]:
df.shape

(200000, 3)

In [21]:
print(df.isna().sum())
df_na = df.dropna()

id          0
document    8
label       0
dtype: int64


1-3. 데이터 전처리(함수로 구현)


*   정제 및 정규화(정규식사용), 어간/표제어처리, 불용어 제거




In [22]:
# 불용어 불러오기
with open('stopwords-ko.txt', encoding='utf-8') as f:
    stopwords = set(w.strip() for w in f if w.strip())
len(stopwords)

595

In [23]:
from konlpy.tag import Okt
import re

okt = Okt()
def preprocess_text(text:str):
    text = text.lower()
    text = re.sub(r'[^0-9a-zA-Z가-힣\s]',' ', text).strip() # 정규식 사용 및 양끝 공백 처리
    text = re.sub(r'\s+', ' ', text) # 연속되는 공백 처리

    morphs = okt.morphs(text, norm=True, stem=True) # 어간,표제어 처리
    return [w for w in morphs if w not in stopwords] # 불용어 제거

## 2. 학습/테스트 데이터 분리

- `train_test_split`으로 데이터를 분리
- 가능하면 `stratify=df["label"]` 옵션을 사용해 **레이블 비율을 유지**


In [25]:
# 층화 추출
df_sample, _ = train_test_split(df_na, train_size=10000, stratify=df_na['label'], random_state=42)

# train/test 분할
X = df_sample['document']
y = df_sample['label']
x_tr, x_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
(len(x_tr), len(x_te))

(8000, 2000)

## 3. 공통 함수: 모델 학습 & 평가

- TF-IDF + 분류기를 하나의 `Pipeline`으로 묶어서 사용
- `classification_report`와 `macro-F1` 점수를 함께 출력


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

lr_ct_clf = Pipeline(steps=[
    ('vect', CountVectorizer(
        tokenizer=preprocess_text,
        token_pattern=None,
        lowercase=False
    )),
    ('model', LogisticRegression(max_iter=1000))
])

nb_ct_clf = Pipeline(steps=[
    ('vect', CountVectorizer(
        tokenizer=preprocess_text,
        token_pattern=None,
        lowercase=False
    )),
    ('model', MultinomialNB(alpha=1.0))
])

svm_ct_clf = Pipeline(steps=[
    ('vect', CountVectorizer(
        tokenizer=preprocess_text,
        token_pattern=None,
        lowercase=False
    )),
    ('model', LinearSVC())
])


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

lr_tfidf_clf = Pipeline(steps=[
    ('vect', TfidfVectorizer(
        tokenizer=preprocess_text,
        token_pattern=None,
        lowercase=False
    )),
    ('model', LogisticRegression(max_iter=1000))
])

nb_tfidf_clf = Pipeline(steps=[
    ('vect', TfidfVectorizer(
        tokenizer=preprocess_text,
        token_pattern=None,
        lowercase=False
    )),
    ('model', MultinomialNB(alpha=1.0))
])

svm_tfidf_clf = Pipeline(steps=[
    ('vect', TfidfVectorizer(
        tokenizer=preprocess_text,
        token_pattern=None,
        lowercase=False
    )),
    ('model', LinearSVC())
])

In [30]:
# 파라미터 설정
lr_param_grid = {
    'vect__ngram_range': [(1,1), (1,2)],
    'vect__min_df': [1,2,3],
    'model__C': [0.1, 0.5, 1.0, 10.0]
}

nb_param_grid = {
    'vect__ngram_range': [(1,1), (1,2)],
    'vect__min_df': [1,2,3],
    'model__alpha': [0.1, 0.5, 1.0, 2.0]
}

svm_param_grid = {
    'vect__ngram_range': [(1,1), (1,2)],
    'vect__min_df': [1,2,3],
    'model__C': [0.1, 0.5, 1.0, 10.0]
}

## 4. 모델별 학습 & 평가

세 가지 모델을 모두 학습해 보고 성능을 비교

1. 로지스틱 회귀 (`LogisticRegression`)
2. 나이브 베이즈 (`MultinomialNB`)
3. LinearSVC (`LinearSVC`)


In [32]:
from sklearn.model_selection import GridSearchCV

gs_lr = GridSearchCV(
    lr_ct_clf,
    param_grid=lr_param_grid,
    scoring="f1_macro", # 불균형 데이터 고려하면 macro-F1 추천
    cv=3,
    n_jobs=1 # okt를 쓰게되면 병렬처리가 불가능, 미리 전처리하고 기본 토크나이저를 쓰면 해결가능
)

gs_lr.fit(x_tr, y_tr)
print("== LogisticRegression + CountVectorizer ==")
print("Best params:", gs_lr.best_params_)
print("Best macro-F1 (cv):", gs_lr.best_score_)

best_lr = gs_lr.best_estimator_
y_pred = best_lr.predict(x_te)
print(classification_report(y_te, y_pred, digits=3))

== LogisticRegression + TF-IDF ==
Best params: {'model__C': 0.5, 'vect__min_df': 1, 'vect__ngram_range': (1, 2)}
Best macro-F1 (cv): 0.7972174571236573
              precision    recall  f1-score   support

           0      0.823     0.808     0.815      1000
           1      0.811     0.826     0.819      1000

    accuracy                          0.817      2000
   macro avg      0.817     0.817     0.817      2000
weighted avg      0.817     0.817     0.817      2000



In [35]:
# 커스텀 전처리 없이
lr_ct_clf = Pipeline(steps=[
    ('vect', CountVectorizer(ngram_range=(1,2), min_df=1)),
    ('model', LogisticRegression(max_iter=1000))
])

nb_ct_clf = Pipeline(steps=[
    ('vect', CountVectorizer(ngram_range=(1,2), min_df=1)),
    ('model', MultinomialNB(alpha=1.0))
])

svm_ct_clf = Pipeline(steps=[
    ('vect', CountVectorizer(ngram_range=(1,2), min_df=1)),
    ('model', LinearSVC())
])

lr_tfidf_clf = Pipeline(steps=[
    ('vect', TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ('model', LogisticRegression(max_iter=1000))
])

nb_tfidf_clf = Pipeline(steps=[
    ('vect', TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ('model', MultinomialNB(alpha=1.0))
])

svm_tfidf_clf = Pipeline(steps=[
    ('vect', TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ('model', LinearSVC())
])

In [41]:
lr_ct_clf.fit(x_tr, y_tr)
lr_ct_pred = lr_ct_clf.predict(x_te)
print('== LogisticRegression + CountVectorizer(전처리X) ==')
print(classification_report(y_te, lr_ct_pred, digits=3))
print()
nb_ct_clf.fit(x_tr, y_tr)
nb_ct_pred = nb_ct_clf.predict(x_te)
print('== NaiveBayes + CountVectorizer(전처리X) ==')
print(classification_report(y_te, nb_ct_pred, digits=3))
print()
svm_ct_clf.fit(x_tr, y_tr)
svm_ct_pred = svm_ct_clf.predict(x_te)
print('== LinearSVC + CountVectorizer(전처리X) ==')
print(classification_report(y_te, svm_ct_pred, digits=3))
print()

== LogisticRegression + CountVectorizer(전처리X) ==
              precision    recall  f1-score   support

           0      0.742     0.774     0.758      1000
           1      0.764     0.731     0.747      1000

    accuracy                          0.752      2000
   macro avg      0.753     0.752     0.752      2000
weighted avg      0.753     0.752     0.752      2000


== NaiveBayes + CountVectorizer(전처리X) ==
              precision    recall  f1-score   support

           0      0.769     0.759     0.764      1000
           1      0.762     0.772     0.767      1000

    accuracy                          0.765      2000
   macro avg      0.766     0.766     0.765      2000
weighted avg      0.766     0.765     0.765      2000


== LinearSVC + CountVectorizer(전처리X) ==
              precision    recall  f1-score   support

           0      0.807     0.658     0.725      1000
           1      0.711     0.843     0.772      1000

    accuracy                          0.750      2

In [42]:
lr_tfidf_clf.fit(x_tr, y_tr)
lr_tfidf_pred = lr_tfidf_clf.predict(x_te)
print('== LogisticRegression + TfidfVectorizer(전처리X) ==')
print(classification_report(y_te, lr_tfidf_pred, digits=3))
print()
nb_tfidf_clf.fit(x_tr, y_tr)
nb_tfidf_pred = nb_tfidf_clf.predict(x_te)
print('== NaiveBayes + TfidfVectorizer(전처리X) ==')
print(classification_report(y_te, nb_tfidf_pred, digits=3))
print()
svm_tfidf_clf.fit(x_tr, y_tr)
svm_tfidf_pred = svm_tfidf_clf.predict(x_te)
print('== LinearSVC + TfidfVectorizer(전처리X) ==')
print(classification_report(y_te, svm_tfidf_pred, digits=3))
print()

== LogisticRegression + TfidfVectorizer(전처리X) ==
              precision    recall  f1-score   support

           0      0.742     0.766     0.754      1000
           1      0.758     0.734     0.746      1000

    accuracy                          0.750      2000
   macro avg      0.750     0.750     0.750      2000
weighted avg      0.750     0.750     0.750      2000


== NaiveBayes + TfidfVectorizer(전처리X) ==
              precision    recall  f1-score   support

           0      0.766     0.771     0.769      1000
           1      0.770     0.765     0.767      1000

    accuracy                          0.768      2000
   macro avg      0.768     0.768     0.768      2000
weighted avg      0.768     0.768     0.768      2000


== LinearSVC + TfidfVectorizer(전처리X) ==
              precision    recall  f1-score   support

           0      0.753     0.773     0.763      1000
           1      0.767     0.746     0.756      1000

    accuracy                          0.759      2

## 5. 성능 비교 표 만들기

세 모델의 macro-F1 점수를 하나의 표로 정리


In [46]:
lr_ct_f1 = f1_score(y_te, lr_ct_pred, average='macro')
nb_ct_f1 = f1_score(y_te, nb_ct_pred, average='macro')
svm_ct_f1 = f1_score(y_te, svm_ct_pred, average='macro')
lr_tfidf_f1 = f1_score(y_te, lr_tfidf_pred, average='macro')
nb_tfidf_f1 = f1_score(y_te, nb_tfidf_pred, average='macro')
svm_tfidf_f1 = f1_score(y_te, svm_tfidf_pred, average='macro')

model_names = ['lr_ct', 'nb_ct', 'svm_ct', 'lr_tfidf', 'nb_tfidf', 'svm_tfidf']
f1_scores = [lr_ct_f1, nb_ct_f1, svm_ct_f1, lr_tfidf_f1, nb_tfidf_f1, svm_tfidf_f1]

df_metrics = pd.DataFrame([f1_scores], index=['macro_f1'], columns=model_names)
df_metrics = df_metrics.applymap(lambda x: f"{x:.4f}") # map은 데이터프레임에 못씀.
print(df_metrics)

           lr_ct   nb_ct  svm_ct lr_tfidf nb_tfidf svm_tfidf
macro_f1  0.7524  0.7655  0.7483   0.7499   0.7680    0.7595


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20076\617810754.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_metrics = df_metrics.applymap(lambda x: f"{x:.4f}") # map은 데이터프레임에 못씀.




### 6. 나이브 베이즈: 클래스별 대표 단어

- `MultinomialNB`의 `feature_log_prob_`를 이용해
- 각 클래스에서 중요한 단어 TOP-N을 뽑기


In [49]:
vect = nb_ct_clf.named_steps['vect']
nb = nb_ct_clf.named_steps['model']

feature_names = np.array(vect.get_feature_names_out())
print(feature_names)

for i, c in enumerate(nb.classes_):
    print(f'===클래스 {c} 대표 단어===')
    log_prob = nb.feature_log_prob_[i]
    top10_idx = log_prob.argsort()[-10:] # 상위 10등
    print(feature_names[top10_idx])

['000' '000 000' '000 000점' ... '힙합을' '힙합을 능욕하는' '힛힛']
===클래스 0 대표 단어===
['없고' '이게' '이건' '영화는' '이런' '정말' '그냥' '진짜' '너무' '영화']
===클래스 1 대표 단어===
['드라마' '영화를' '보고' 'ㅋㅋ' '이런' '최고의' '진짜' '너무' '정말' '영화']


In [50]:
vect = nb_tfidf_clf.named_steps['vect']
nb = nb_tfidf_clf.named_steps['model']

feature_names = np.array(vect.get_feature_names_out())
print(feature_names)

for i, c in enumerate(nb.classes_):
    print(f'===클래스 {c} 대표 단어===')
    log_prob = nb.feature_log_prob_[i]
    top10_idx = log_prob.argsort()[-10:] # 상위 10등
    print(feature_names[top10_idx])

['000' '000 000' '000 000점' ... '힙합을' '힙합을 능욕하는' '힛힛']
===클래스 0 대표 단어===
['이건' '이런' '아깝다' '이게' '정말' '쓰레기' '그냥' '진짜' '너무' '영화']
===클래스 1 대표 단어===
['재밌게' '드라마' '재밌어요' 'ㅋㅋ' '최고의' '최고' '진짜' '너무' '정말' '영화']


### 7. LinearSVC: 단어 가중치 분석

- `coef_`를 이용해 각 단어가 어떤 클래스로 기울게 만드는지 확인



In [52]:
vect = svm_ct_clf.named_steps['vect']
clf = svm_ct_clf.named_steps['model']

feature_names = np.array(vect.get_feature_names_out())
coef = clf.coef_ # 값이크면:긍정, 값이작으면:부정, 절댓값이크면:영향력큼
print(coef)
top10_pos = coef[0].argsort()[-10:]
print("== 긍정에 강하게 기여하는 단어 ==")
print(feature_names[top10_pos])

top10_neg = coef[0].argsort()[:10]
print("== 부정에 강하게 기여하는 단어 ==")
print(feature_names[top10_neg])

[[ 0.          0.          0.         ... -0.06051    -0.06051
  -0.29379038]]
== 긍정에 강하게 기여하는 단어 ==
['좋다' '재밌음' '재밌네' '최고다' '좋아요' '재밌습니다' '재밋다' '재미있어요' '재밌어요' '최고']
== 부정에 강하게 기여하는 단어 ==
['재미없음' '최악' '재미없다' '쓰레기' '재미없어' '별루' '지루하다' '별로' '쓰레기영화' '너무 재미있다']


In [53]:
vect = svm_tfidf_clf.named_steps['vect']
clf = svm_tfidf_clf.named_steps['model']

feature_names = np.array(vect.get_feature_names_out())
coef = clf.coef_ # 값이크면:긍정, 값이작으면:부정, 절댓값이크면:영향력큼
print(coef)
top10_pos = coef[0].argsort()[-10:]
print("== 긍정에 강하게 기여하는 단어 ==")
print(feature_names[top10_pos])

top10_neg = coef[0].argsort()[:10]
print("== 부정에 강하게 기여하는 단어 ==")
print(feature_names[top10_neg])

[[ 0.25178405  0.16785603  0.08392802 ... -0.16223922 -0.16223922
  -0.37359538]]
== 긍정에 강하게 기여하는 단어 ==
['재밌게' '그리고' '다시' '눈물이' '명작' '좋다' '최고다' '10점' '최고' '최고의']
== 부정에 강하게 기여하는 단어 ==
['아깝다' '쓰레기' '최악의' 'ㅡㅡ' '없고' '지루하다' '최악' '스토리' '없는' '뭐야']


# 결과 분석

macro_f1 기준 nNaive Bayes와 TfidfVectorizer 조합이 0.7680으로 좋은 성능을 보였고 그 다음으로 Naive Bayes와 CountVectorizer가 좋은 성능을 보였다.
따라서 Naive Bayes가 이 데이테에 대해서 가장 좋은 성능을 가지는 모델이라고 볼 수 있다.

시간이 부족하여 전처리를 하지 못했는데, 때문에 '--', 'ㅋㅋ' 이런 정제되지 않은 단어들이 포함된 것을 확인할 수 있었다.
전처리를 했으면 또 다른 결과가 나왔을 수도 있고 gridsearch를 통해 파라미터 튜닝을 한다면 더 좋은 성능이 나올 것으로 기대한다.

          lr_ct   nb_ct   svm_ct  lr_tfidf nb_tfidf svm_tfidf
macro_f1  0.7524  0.7655  0.7483   0.7499   0.7680    0.7595